In [ ]:
!pip install requests
import requests
!pip install beautifulsoup4
from bs4 import BeautifulSoup
from csv import writer
import os

In [ ]:
# Get list of all industries
url_industries = "https://www.messen.de/de/branchen"

page = requests.get(url_industries)
soup = BeautifulSoup(page.content, 'html.parser')

if False:
    # Example
    """
    <ul class="nav nav-tabs nav-tabs-index">
        # letter selector
    <div class="tab-content tab-content-index">
        <ul class="nav nav-pills nav-stacked">
            <li>
                <a href="/de/1586/branche/50plus" title="Messen 50plus">
                    #50plus
                </a>

    nachrichten = soup.findAll('ul', {'class':'list'})
    links = []
    for ul in nachrichten:
        links.extend(ul.findAll('a'))
    print len(links)
    """

industries = []

industries_list = soup.findAll('ul', {'class':'nav-pills'})
for ind in industries_list:
    for a in ind.findAll('a', href=True):
        industry_name = a.contents[0]
        industry_url = a['href']
        # print(industry_name, industry_url)
        industries.append("https://messen.de" + industry_url)

# print(industries)

In [ ]:
# Iterate through all industries

tradefair_overview_urls = []

# Open each industry page

for industry in industries:
# Example ID: Automessen 22

    page = requests.get(industry)
    soup = BeautifulSoup(page.content, 'html.parser')
    # print(soup.prettify())

    pagination = soup.find('ul', {'class':'pagination'})
    if pagination == None:
        max_page = 1
    else:
        page_num = ""
        for num in pagination.findAll('a'):
            if num.contents[0] != "»":
                page_num = num.contents[0]
                # print(page_num)
        max_page = int(page_num)

    #print(max_page)

        
    # Find number of pages
    # example page 01: https://www.messen.de/de/1416/branche/bau-und-heimwerkerbedarf
    # example page 02: https://www.messen.de/de/1416/branche/bau-und-heimwerkerbedarf?offset=25
    # example page 10: https://www.messen.de/de/1416/branche/bau-und-heimwerkerbedarf?offset=225  (last page)
    # offset = (page_number - 1) * 25

    industry_pages = [industry]
    for i in range(max_page):
        offset = i * 25
        # print(offset)
        if i > 0:
            industry_pages.append(industry + "?offset=" + str(offset))
    
    # print(industry_pages)
    tradefair_overview_urls.extend(industry_pages)
    
    # better external save in case messen.de caps off access
    for ind_page in industry_pages:
        print(ind_page)
        with open('tradefair_overview_urls.csv','a') as f:
            f.write(ind_page + "\n")

In [ ]:
# Iterate through all pages

import re

def get_messen(messen_url):

    industry = re.search("branche\/(.+?)\/?(?:\?offset|$)", messen_url).group(1)

    page = requests.get(messen_url)
    soup = BeautifulSoup(page.content, 'html.parser')

    messen = soup.findAll('div', {'class':'messe'})

    messe_list = []
    for messe in messen:
        # print(messe.prettify())

        messe_name = ""
        messe_url = ""
        messe_img = ""
        messe_date = ""
        messe_dur = ""
        messe_loc = ""
        messe_sub = ""


        if messe.find('div', {'class':'messename'}).find("a").find("span"):
            messe_name = messe.find('div', {'class':'messename'}).find("a").find("span").text

        if messe.find('div', {'class':'messename'}).find("a"):
            messe_url = "https://messen.de" + messe.find('div', {'class':'messename'}).find("a")['href']
        
        if messe.find('div', {'class':'messelogo'}).find("a").find("img"):
            messe_img = messe.find('div', {'class':'messelogo'}).find("a").find("img")['src']
        
        if messe.find('div', {'class':'messedatum'}).find('div', {'class':'unbekannt'}):
            if messe.find('div', {'class':'messedatum'}).find('div', {'class':'unbekannt'}).text == "nächster Termin steht noch nicht fest":
                messe_date = "nächster Termin steht noch nicht fest"
            else: messe_date = "unbekannt"
        else:
            if messe.find('div', {'class':'datum'}):
                messe_date = messe.find('div', {'class':'datum'}).text

            if messe.find('div', {'class':'laufzeit'}):
                messe_dur = messe.find('div', {'class':'laufzeit'}).text
            
            if messe.find('div', {'class':'termin-abgesagt-hinweis'}):
                messe_dur = messe.find('div', {'class':'termin-abgesagt-hinweis'}).text
            
        if messe.find('div', {'class':'messeort'}):
            messe_loc = messe.find('div', {'class':'messeort'}).text
        
        if messe.find('div', {'class':'messeuntertitel'}):
            messe_sub = messe.find('div', {'class':'messeuntertitel'}).text # .text is better than .contents[0] as it handles zero-strings better

        messe_list.append([messe_name, messe_loc, messe_date, messe_dur, messe_sub, messe_url, industry, messe_img])
        
    # print(messe_list)

    if True:
        # As backup append to .csv file
        for mes in messe_list:
            # print(mes)
            with open('tradefair_overview.csv','a') as f:
                wr = csv.writer(f, dialect='excel')
                wr.writerow(mes)

In [ ]:
for url in tradefair_overview_urls:
    get_messen(url)
# bei ausbildungsmessen unterbrochen

In [ ]:
# in case requests were interupted, this helps you continue from a certain point
for url in tradefair_overview_urls[105:200]:
    get_messen(url)


os.system('say "your program has finished"')

In [ ]:
# in case requests were interupted, this helps you continue from a certain point
# with timer

start = time.time()
with open('tradefair_overview.csv','r') as f:
    count = sum(1 for line in f)

for url in tradefair_overview_urls[1200:]:
    get_messen(url)

end = time.time()
os.system('say "your program has finished"')

with open('tradefair_overview.csv','r') as f:
    print(sum(1 for line in f) - count, " rows added in ", (end-start)/60, "minutes.")


In [ ]:
len(tradefair_overview_urls)